In [1]:
import numpy as np
from sklearn.mixture import GMM
from sklearn.cross_validation import StratifiedKFold
import itertools
from scipy import linalg

import ubm_adapt

In [2]:
mfcc_original_train = np.loadtxt('mfcc_original_train.txt')

mfcc_spoof_train = np.loadtxt('mfcc_spoof_train.txt')

mfcc_original_develop = np.loadtxt('mfcc_original_dev.txt')

mfcc_spoof_develop = np.loadtxt('mfcc_spoof_dev.txt')

mfcc_original_test = np.loadtxt('mfcc_original_eva.txt')

mfcc_spoof_test = np.loadtxt('mfcc_spoof_eva.txt')

# Train set

In [3]:
X_train = np.concatenate([mfcc_original_train, mfcc_spoof_train]) 
y_train = np.concatenate([np.zeros((mfcc_original_train.shape[0])), np.ones((mfcc_spoof_train.shape[0]))]).astype('int')

# Development set 

In [4]:
X_develop = np.concatenate([mfcc_original_develop, mfcc_spoof_develop]) 
y_develop = np.concatenate([np.zeros((mfcc_original_develop.shape[0])), np.ones((mfcc_spoof_develop.shape[0]))]).astype('int')


# Test set

In [5]:
X_test = np.concatenate([mfcc_original_test, mfcc_spoof_test]) 
y_test = np.concatenate([np.zeros((mfcc_original_test.shape[0])), np.ones((mfcc_spoof_test.shape[0]))]).astype('int')


In [6]:
def computeUBM(ubm_model, data):
    ###########################################
    # ubm_model - gmm-represent distribution of our model
    # data - samples, which will correct ubm-model
    ###############################################

    xdim = data.shape[1]
    M = ubm_model.n_components

    ###############################################################
    #    ubm_means: means of the ubm <number array>               #
    #    ubm_covars: covariances of the ubm <number array>        #
    #    ubm_weights: weights of the ubm <number array>           #
    #    new_means: means adapted from the ubm <number array>     #
    #    new_weights: weights adapted from the ubm <number array> #
    ###############################################################

    # Copy parameters GMM-model
    ubm_weights = ubm_model.weights_
    ubm_means = ubm_model.means_
    ubm_covars = ubm_model.covars_

    ###################################################################
    # for X = {x_1, ..., x_T}                                         #
    # P(i|x_t) = w_i * p_i(x_t) / sum_j=1_M(w_j * P_j(x_t))           #
    ###################################################################

    posterior_prob = ubm_model.predict_proba(data)
    pr_i_xt = (ubm_weights * posterior_prob) / np.asmatrix(np.sum(ubm_weights \
                                                                  * posterior_prob, axis=1)).T

    n_i = np.asarray(np.sum(pr_i_xt, axis=0)).flatten()  # [M, ]

    # Then we can compute E(x) and E(x2) and calculate new parameters of
    # our model

    E_x = np.asarray([(np.asarray(pr_i_xt[:, i]) * data).sum(axis=0) / n_i[i] if not n_i[i] == 0. else np.zeros(xdim) for i in range(M)])  # [M x xdim]
    E_x2 = np.asarray([(np.asarray(pr_i_xt[:, i]) * (data ** 2)).sum(axis=0) / n_i[i]  if not n_i[i] == 0. else np.zeros(xdim)for i in range(M)])  # [M x xdim]

    ################################################################
    #    T: scaling factor, number of samples                      #
    #    relevance_factor: factor for scaling the adapted means    #
    #    scaleparam - scale parameter for weights matrix estimation#
    ################################################################

    T = data.shape[0]
    relevance_factor = 16
    scaleparam = 1

    ################################################################
    # compute alpha_i: data-depentend apaptation coefficient       #
    # alpha_w = alpha_m = alpha_v                                  #
    # alpha_i = n_i/ (n_i + relevance factor)                      #
    ################################################################

    alpha_i = n_i / (n_i + relevance_factor)

    ###############################
    # Parqameter`s adaptation
    ##############################
    new_weights = (alpha_i * n_i / T + (1.0 - alpha_i) * ubm_weights) * scaleparam

    alpha_i = np.asarray(np.asmatrix(alpha_i).T)
    new_means = (alpha_i * E_x + (1. - alpha_i) * ubm_means)
    new_covars = alpha_i * E_x2 + (1. - alpha_i) * (ubm_covars + (ubm_means ** 2)) - (new_means ** 2)

    #############################################
    # if we want compute `full` covariance matrix - comment code here
    # new_covars = np.zeros([M, xdim, xdim])
    # for j in range(M):
    #    new_covars[j] = alpha_i[j]*E_x2[j] +(1. - alpha_i[j]).flatten()*(ubm_covars[j] + (new_means[j]**2))- (ubm_means[j]**2)
    #    new_covars[i] = np.where(new_covars[i] < MIN_VARIANCE, MIN_VARIANCE, new_covars[i])
    ####################################################################
    #   `covars_` : array
    #    Covariance parameters for each mixture component.  The shape
    #    depends on `covariance_type`::
    #        (n_components, n_features)             if 'spherical',
    #        (n_features, n_features)               if 'tied',
    #        (n_components, n_features)             if 'diag',
    #        (n_components, n_features, n_features) if 'full'
    #####################################################################

    ubm_model.means_ = new_means
    ubm_model.weights_ = new_weights
    ubm_model.covars_ = new_covars

    return ubm_model

# Experiment

In [7]:
n_g = 1024
g1 =  GMM(n_components = n_g, covariance_type='diag',init_params='wmc', n_iter=20)
g1.fit(mfcc_original_train)

g2 =  GMM(n_components = n_g, covariance_type='diag',init_params='wmc', n_iter=20)
g2.fit(mfcc_spoof_train)

prediction  = np.array(np.log(mfcc_original_train.shape[0])+ g1.score(X_develop)  < np.log(mfcc_spoof_train.shape[0])+g2.score(X_develop)).astype('int')

accuracy = np.mean(prediction == y_develop) * 100
print 'accuracy - prediction on development set', accuracy


accuracy - prediction on development set 78.692947613


In [8]:
prediction  = np.array(np.log(mfcc_original_train.shape[0])+ g1.score(X_test)  < np.log(mfcc_spoof_train.shape[0])+g2.score(X_test)).astype('int')

accuracy = np.mean(prediction == y_test) * 100
print 'accuracy - prediction on evaluation set', accuracy

accuracy - prediction on evaluation set 76.8050298856


In [9]:
g1_adapt = computeUBM(g1, mfcc_original_develop)
g2_adapt = computeUBM(g2, mfcc_spoof_develop)

In [10]:
prediction  = np.array(np.log(mfcc_original_train.shape[0])+ g1_adapt.score(X_test)  < np.log(mfcc_spoof_train.shape[0])+g2_adapt.score(X_test)).astype('int')
accuracy = np.mean(prediction == y_test) * 100
print 'accuracy - prediction on evaluation set. adapted on development', accuracy


accuracy - prediction on evaluation set. adapted on development 87.720005791


In [ ]:
acc = []
for i in range(1,10):
    g1_adapt = computeUBM(g1_adapt, mfcc_original_develop)
    g2_adapt = computeUBM(g2_adapt, mfcc_spoof_develop)
    
    prediction  = np.array(np.log(mfcc_original_train.shape[0])+ g1_adapt.score(X_test)  < np.log(mfcc_spoof_train.shape[0])+g2_adapt.score(X_test)).astype('int')
    accuracy = np.mean(prediction == y_test) * 100
    print accuracy
    acc.append(accuracy)
    
print np.max(acc)